In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [15]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

n_docs = len(documents)

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
n_words = len(dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]
print("n_docs", n_docs)
print('n_words', n_words)
corpus

2021-03-22 13:38:22,242 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-03-22 13:38:22,244 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


n_docs 9
n_words 12


[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [6]:
from gensim import models

tfidf = models.TfidfModel(corpus)

2021-03-22 13:15:51,029 : INFO : collecting document frequencies
2021-03-22 13:15:51,030 : INFO : PROGRESS: processing document #0
2021-03-22 13:15:51,031 : INFO : calculating IDF weights for 9 documents and 12 features (28 matrix non-zeros)


TypeError: 'int' object is not iterable

In [8]:
doc_bow = [(0,1), (1,1)]
print(tfidf[doc_bow])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [21]:
corpus_tfidf = tfidf[corpus]
print(len(corpus_tfidf))
for doc in corpus_tfidf:
    print(doc)

9
[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [10]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi_model[corpus_tfidf]

2021-03-22 13:33:06,391 : INFO : using serial LSI version on this node
2021-03-22 13:33:06,397 : INFO : updating model with new documents
2021-03-22 13:33:06,399 : INFO : preparing a new chunk of documents
2021-03-22 13:33:06,406 : INFO : using 100 extra samples and 2 power iterations
2021-03-22 13:33:06,407 : INFO : 1st phase: constructing (12, 102) action matrix
2021-03-22 13:33:06,417 : INFO : orthonormalizing (12, 102) action matrix
2021-03-22 13:33:06,424 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2021-03-22 13:33:06,428 : INFO : computing the final decomposition
2021-03-22 13:33:06,441 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2021-03-22 13:33:06,458 : INFO : processed documents up to #9
2021-03-22 13:33:06,461 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2021-03-22 13:33:06,462 : INFO : topic #

In [20]:
lsi_model.print_topics(2)
print(len(corpus_lsi))  # components of the docs in the subspace

2021-03-22 13:46:18,576 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2021-03-22 13:46:18,578 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


9


In [19]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090293), (1, -0.5200703306361854)] Human machine interface for lab abc computer applications
[(0, 0.1966759285914238), (1, -0.760956316770006)] A survey of user opinion of computer system response time
[(0, 0.08992639972446402), (1, -0.7241860626752513)] The EPS user interface management system
[(0, 0.07585847652178154), (1, -0.632055158600343)] System and human system engineering testing of EPS
[(0, 0.1015029918498003), (1, -0.5737308483002965)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378318), (1, 0.16115180214025693)] The generation of random binary unordered trees
[(0, 0.8774787673119836), (1, 0.1675890686465928)] The intersection graph of paths in trees
[(0, 0.9098624686818578), (1, 0.14086553628718884)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569275), (1, -0.053929075663894696)] Graph minors A survey


In [22]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name) # same for other models

loaded_lsi_model = models.LsiModel.load(tmp.name)
os.unlink(tmp.name)

2021-03-22 13:52:49,796 : INFO : saving Projection object under /var/folders/sc/2_7p0t5d1j98c77km1h428nw0000gq/T/model-zjewsdd3.lsi.projection, separately None
2021-03-22 13:52:49,809 : INFO : saved /var/folders/sc/2_7p0t5d1j98c77km1h428nw0000gq/T/model-zjewsdd3.lsi.projection
2021-03-22 13:52:49,810 : INFO : saving LsiModel object under /var/folders/sc/2_7p0t5d1j98c77km1h428nw0000gq/T/model-zjewsdd3.lsi, separately None
2021-03-22 13:52:49,810 : INFO : not storing attribute projection
2021-03-22 13:52:49,811 : INFO : not storing attribute dispatcher
2021-03-22 13:52:49,812 : INFO : saved /var/folders/sc/2_7p0t5d1j98c77km1h428nw0000gq/T/model-zjewsdd3.lsi
2021-03-22 13:52:49,813 : INFO : loading LsiModel object from /var/folders/sc/2_7p0t5d1j98c77km1h428nw0000gq/T/model-zjewsdd3.lsi
2021-03-22 13:52:49,816 : INFO : loading id2word recursively from /var/folders/sc/2_7p0t5d1j98c77km1h428nw0000gq/T/model-zjewsdd3.lsi.id2word.* with mmap=None
2021-03-22 13:52:49,816 : INFO : setting ignore